In [1]:
import jieba

tokenizer_word = lambda x: [y for y in jieba.lcut(x)]  # 以词为单位构建词表(用jieba分词)
tokenizer_char = lambda x: [y for y in x]  # 以字为单位构建词表


# 句子转标号
def str2index(string, tokenizer, vocab):
    index = tokenizer(string)
    return [vocab.index(idx) if idx in vocab else len(vocab) - 1 for idx in index]

In [2]:
string = '我今天去你家吃饭'
vocab_word = ['我', '今天', '吃', '的', '很', '开心', '去', '你家', '[UKN]']
vocab_char = ['我', '今', '天', '吃', '的', '很', '开', '心', '去', '家', '[UKN]']

indexByWord = str2index(string, tokenizer_word, vocab_word)
indexByChar = str2index(string, tokenizer_char, vocab_char)

print(indexByWord)
print(indexByChar)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ldy\AppData\Local\Temp\jieba.cache
Loading model cost 0.484 seconds.
Prefix dict has been built successfully.


[0, 1, 6, 7, 8]
[0, 1, 2, 8, 10, 9, 3, 10]


In [3]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn

# 用预训练的BERT模型进行初始化
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')
# 输入句子
sentence = "这是一个示例句子。"
# 使用BERT Tokenizer对句子进行编码
input_ids = tokenizer.encode(text=sentence)
input_ids = torch.LongTensor(input_ids).unsqueeze(0)

# 使用BERT模型获取词嵌入
with torch.no_grad():
    last_hidden_states,cls_states = model(input_ids, return_dict=False)
lc = nn.Linear(768, 5)
out = lc(cls_states)

# print(last_hidden_states)
print(input_ids) # tokenizer后的句子张量
print(last_hidden_states.shape) # bert输出的句子embedding张量，shape第一维表示句子，第二维表示token，第三维表示token的特征维度
print(cls_states.shape) # bert输出的[CLS]的张量，shape第一维表示句子，第二维表示特征维度
print(out.shape) # [CLS]张量经过全连接层后的输出，输出每一个label的概率

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 6821, 3221,  671,  702, 4850,  891, 1368, 2094,  511,  102]])
torch.Size([1, 11, 768])
torch.Size([1, 768])
torch.Size([1, 5])
